# Inference

In [92]:
import os
import json
import torch
import argparse

from model_bowloss import SentenceVAE
from utils import to_var, idx2word, interpolate, AttributeDict

In [93]:
args = {
    'load_checkpoint': './bin/2019-Nov-29-07:07:02/E9.pytorch',
    'num_samples': 10,
    'max_sequence_length': 50,
    'embedding_size': 300,
    'rnn_type': 'gru',
    'hidden_size': 256,
    'word_dropout': 0,
    'embedding_dropout': 0.5,
    'latent_size': 16,
    'num_layers': 1,
    'bidirectional': False
}
args = AttributeDict(args)
args.rnn_type = args.rnn_type.lower()
assert args.rnn_type in ['rnn', 'lstm', 'gru']
assert 0 <= args.word_dropout <= 1

## utils

In [94]:
def to_tensor(arr_like, cuda=True):
    tensor = torch.Tensor(arr_like)
    return tensor if not cuda else tensor.cuda()

## load vocab

In [95]:
with open('./data/eccos/src/ptb.vocab.json', 'r') as file:
        src_vocab = json.load(file)
src_w2i, src_i2w = src_vocab['w2i'], src_vocab['i2w']

In [96]:
with open('./data/eccos/tgt/ptb.vocab.json', 'r') as file:
        tgt_vocab = json.load(file)
tgt_w2i, tgt_i2w = tgt_vocab['w2i'], tgt_vocab['i2w']

In [97]:
len(src_w2i), len(tgt_w2i)

(5619, 12106)

In [7]:
# args.obj['cond_embedding_size'] = 300
# args.obj['cond_hidden_size'] = 256

## load model

In [8]:
from ptb import SOS_INDEX, EOS_INDEX, PAD_INDEX, UNK_INDEX

In [9]:
model = SentenceVAE(
    vocab_size=len(src_w2i),
    out_vocab_size=len(tgt_w2i),
    sos_idx=SOS_INDEX,
    eos_idx=EOS_INDEX,
    pad_idx=PAD_INDEX,
    unk_idx=UNK_INDEX,
    max_sequence_length=args.max_sequence_length,
    embedding_size=args.embedding_size,
    rnn_type=args.rnn_type,
    hidden_size=args.hidden_size,
    word_dropout=args.word_dropout,
    embedding_dropout=args.embedding_dropout,
    latent_size=args.latent_size,
    num_layers=args.num_layers,
    bidirectional=args.bidirectional,
    
    use_bow_loss=False,
#     bow_hidden_size=256,
)

In [10]:
model

SentenceVAE(
  (embedding): Embedding(5619, 300)
  (embedding_dropout): Dropout(p=0.5, inplace=False)
  (decoder_embedding): Embedding(12106, 300)
  (encoder_rnn): GRU(300, 256, batch_first=True)
  (decoder_rnn): GRU(300, 256, batch_first=True)
  (hidden2mean): Linear(in_features=256, out_features=16, bias=True)
  (hidden2logv): Linear(in_features=256, out_features=16, bias=True)
  (latent2hidden): Linear(in_features=16, out_features=256, bias=True)
  (outputs2vocab): Linear(in_features=256, out_features=12106, bias=True)
)

In [11]:
if not os.path.exists(args.load_checkpoint):
    raise FileNotFoundError(args.load_checkpoint)

model.load_state_dict(torch.load(args.load_checkpoint))
print("Model loaded from %s"%(args.load_checkpoint))

if torch.cuda.is_available():
    model = model.cuda()

Model loaded from ./bin/2019-Nov-29-07:07:02/E9.pytorch


In [12]:
model.eval()

SentenceVAE(
  (embedding): Embedding(5619, 300)
  (embedding_dropout): Dropout(p=0.5, inplace=False)
  (decoder_embedding): Embedding(12106, 300)
  (encoder_rnn): GRU(300, 256, batch_first=True)
  (decoder_rnn): GRU(300, 256, batch_first=True)
  (hidden2mean): Linear(in_features=256, out_features=16, bias=True)
  (hidden2logv): Linear(in_features=256, out_features=16, bias=True)
  (latent2hidden): Linear(in_features=16, out_features=256, bias=True)
  (outputs2vocab): Linear(in_features=256, out_features=12106, bias=True)
)

## sample 指定 inference

### load test data

In [13]:
from ptb import PTB
test_src_ptb = PTB(
    data_dir='./data/eccos/src/',
    split='test',
    create_data=False,
)

In [14]:
test_src_ptb

In [15]:
# 実際のデータ確認用
def ids2text(id_list, ptb, sep=''):
    return sep.join([ptb.i2w[f'{i}'] for i in id_list])

In [40]:
def words2ids(words, ptb):
    assert type(words) == list
    return [test_src_ptb.w2i.get(word, UNK_INDEX) for word in words]

def words2sample(words, ptb):
    id_list = [SOS_INDEX] + words2ids(words, ptb)
    sample = {'input': id_list, 'length': len(id_list)}
    return sample

In [88]:
# データをピックアップ, 確認
# sample = words2sample('可愛い ファッション 春'.split(), test_src_ptb)
sample = test_src_ptb.data['10']
print(f'▼ Input length:{sample["length"]}\n{ids2text(sample["input"], test_src_ptb, " ")}')
sample_input = to_tensor(sample['input']).view(1,-1).to(dtype=torch.int64)
sample_length = to_tensor([sample['length']]).to(dtype=torch.int64)

▼ Input length:4
<sos> ランキング 化粧品 人気


### sampling

In [89]:
# %pdb on
# 潜在変数zの取得
mean, logv, z = model.encode(sample_input, sample_length)
samples, _ = model.inference(z=z)
print('■ 入力')
print(*idx2word(torch.Tensor(sample['input']).int().view(1, -1), i2w=src_i2w, pad_idx=PAD_INDEX), sep='\n')

■ 入力
<sos> ランキング 化粧品 人気


In [90]:
print('■ 一文サンプリング')
print(*[s.replace('<eos>', '').replace(' ', '') for s in idx2word(samples, i2w=tgt_i2w, pad_idx=PAD_INDEX)], sep='\n')

■ 一文サンプリング
【<num>年版】のおすすめコスメ<num>選|domani


In [98]:
# n回サンプリング
args.num_samples = 30
n_samples = [model.encode(sample_input, sample_length) for _ in range(args.num_samples)]
z = torch.cat([z for mean, logv, z in n_samples])
samples, _ = model.inference(z=z)
print('■ 入力')
print(*idx2word(torch.Tensor(sample['input']).int().view(1, -1), i2w=src_i2w, pad_idx=PAD_INDEX), sep='\n')
print(f'■ {args.num_samples}件 サンプリング')
print(*[s.replace('<eos>', '').replace(' ', '') for s in idx2word(samples, i2w=tgt_i2w, pad_idx=PAD_INDEX)], sep='\n')

■ 入力
<sos> ランキング 化粧品 人気
■ 30件 サンプリング
知ってる?知ってる?人気の「<unk>」がヤバい!
美容効果が期待できる!「<unk>」のおすすめアイテム<num>選
男ウケ抜群!「<unk>」の意外な用途になる方法
口コミ評価試験!口コミで人気の「化粧水」まとめ
人気モデルのモデルが誇るの?「<unk>」の魅力とは?
【<num>年人気だった記事をおさらい♡】今年の夏は絶対買うべきはもう古い?
「それどこ」がいいの?若い悩み別におすすめのアイテムを✓
知ってる?知っておきたい知識<num>選
【ランキング】のおすすめコスメ<num>選|domani
人気の韓国ドラマのおすすめ美容室<num>選♪
【<num>年最新版】人気ランキングランキング@cosme
【<num>年版】のおすすめコスメ<num>選|domani
人気の韓国ドラマ、しまむらの<num>年秋冬の美容法をおさらい!
【ランキング】人気ランキングランキング@cosme
【<num>年人気だった記事をおさらい♡】今年のトレンドは、``<unk>"がキテる!
【厳選】人気の香水ランキング@cosme
<num>年福袋の中身ネタバレ
メイク初心者<person>がこぞって使う「<unk>」のコスメが可愛すぎる件
【<num>年人気だった記事をおさらい♡】今年のトレンドは、``<unk>"がキテるらしい。
【<num>円以下】人気のプチプラコスメまとめ
コスパ最強!みんなお勧め「<unk>」が超優秀♡
化粧水の化粧水はこれ!のびるてよかったスキンケア用品が優秀!
【<num>年最新版】人気ランキングランキングランキング@cosme
【<num>年人気だった記事をおさらい♡】今年のトレンドは、``<unk>"がキテる!
化粧品業界の化粧品ブランドまとめ
【プチプラ】アトピーのいい香りでも使える!
【<num>年】人気の育毛剤まとめ
「<unk>」が人気の理由とは?
【<num>】人気のプチプラコスメまとめ♡アットコスメ
化粧品業界のパイオニアで、<unk>を<unk>に<unk>う!


## 潜在空間のプロット
圧縮して分布を確認してみる

In [100]:
import umap.umap_ as umap

In [ ]:
# 描画用
# https://github.com/lmcinnes/umap/blob/master/notebooks/UMAP%20usage%20and%20parameters.ipynb
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns
%matplotlib inline
sns.set(style='white', rc={'figure.figsize':(12,8)})
def plot_scatter(u, title=''):
    fig = plt.figure()
    n_components = u.shape[1]
    plot_kwargs = {'alpha': 0.5, 's':5}
    if n_components == 1:
        ax = fig.add_subplot(111)
        ax.scatter(u[:,0], range(len(u)), **plot_kwargs)
    if n_components == 2:
        ax = fig.add_subplot(111)
        ax.scatter(u[:,0], u[:,1], **plot_kwargs)
    if n_components == 3:
        ax = fig.add_subplot(111, projection='3d', **plot_kwargs)
        ax.scatter(u[:,0], u[:,1], u[:,2])
    plt.title(title, fontsize=18)

### load test data

In [102]:
from ptb import PTB
test_ptb = PTB(
    data_dir='./data/eccos/tgt/',
    split='test',
    create_data=False,
)

In [103]:
def sample_to_tensor(sample):
    sample_input = to_tensor(sample['input']).view(1,-1).to(dtype=torch.int64)
    sample_length = to_tensor([sample['length']]).to(dtype=torch.int64)
    return sample_input, sample_length

In [104]:
%%time
# テストデータを潜在変数に変換
with torch.no_grad():
    encoded_samples = [model.encode(*sample_to_tensor(sample)) for i, sample in test_ptb.data.items()]
    encoded_mean_list = [mean.tolist() for mean, std, z in encoded_samples]

RuntimeError: CUDA error: device-side assert triggered

> /root/user/work/src/Sentence-VAE/model_bowloss.py(101)encode()
     99         input_embedding = self.embedding(input_sequence)
    100 
--> 101         packed_input = rnn_utils.pack_padded_sequence(input_embedding, sorted_lengths.data.tolist(), batch_first=True)
    102 
    103         _, hidden = self.encoder_rnn(packed_input)



ipdb>  q


In [ ]:
encoded_mean_arr = np.array(encoded_mean_list)

In [ ]:
encoded_mean_arr.shape

In [ ]:
%%time
mean_u = umap.UMAP().fit_transform(encoded_mean_arr)

In [99]:
plot_scatter(mean_u)

NameError: name 'plot_scatter' is not defined

> <ipython-input-99-24ec853022e7>(1)<module>()
----> 1 plot_scatter(mean_u)



ipdb>  q


- 分布の形の歪さは、umapの特性によるものなので、なんとも言えない
- ただ、分布の中で偏りはある程度あるので、この偏りの箇所を確認したい
- また、近しい箇所や遠い箇所で意味的な距離がありそうかを確かめたい